# Libraries

In [ ]:
!pip install Sastrawi
!pip install clean-text
!pip install deep-translator

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 209.7/209.7 kB 4.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 175.4/175.4 kB 4.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.4/53.4 kB 6.6 MB/s eta 0:00:00
  Created wheel for emoji: filename=emoji-1.7.0-py3-none-any.whl size=171033 sha256=424fc3b8fc14e654a11d5be1c4028d7217126d026f8bda5dd4a008844065bd0c
  Stored in directory: /root/.cache/pip/wheels/31/8a/8c/315c9e5d7773f74b33d5ed33f075b49c6eaeb7cedbb86e2cf8
Successfully built emoji
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.3/42.3 kB 395.7 kB/s eta 0:00:00


In [ ]:
import pandas as pd
import numpy as np
from nltk.corpus import stopwords
import re
import nltk
nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('punkt')
from nltk.stem import PorterStemmer
from nltk.stem import LancasterStemmer
from nltk.tokenize import word_tokenize
from Sastrawi.Stemmer.StemmerFactory import StemmerFactory
from Sastrawi.StopWordRemover.StopWordRemoverFactory import StopWordRemoverFactory, StopWordRemover, ArrayDictionary
from cleantext import clean
from collections import Counter

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


In [ ]:
import sklearn

In [ ]:
# !pip freeze > requirements.txt

# Get Data from ES and prepare DF

In [ ]:
import requests
import json

In [ ]:
elasticsearch_url = 'https://es-1.datanusantara.com:9201'
username = 'adminkibana'
password = 'kibanaPASS!@#2019'
index_endpoint = 'news-crawl-production-v2'

In [ ]:
#query = {
#    "_source": ["text", "source","source_detail","target"],
 #   "size":600,
  #  "query": {
   #     "match": {
    #              "source": "news"
     #           },
      #  "match": {
       #     "target": "banggai"
  #      }
 #   }
#}

#search_url = f"{elasticsearch_url}/{index_endpoint}/_search"
#response = requests.get(search_url, json=query, auth=(username, password))

In [ ]:
query = {
  "_source": ["target", "title","description_content"],
  "size":600,
  "query": {
    "bool": {
      "must": [],
      "filter": [
        {
          "match_all": {}
        },
        {
          "bool": {
            "should": [


              {
                "match_phrase": {
                  "target": "banggai"
                }
              }
            ],
            "minimum_should_match": 1


          }
        }
      ]
    }
  }
}
search_url = f"{elasticsearch_url}/{index_endpoint}/_search"
response = requests.get(search_url, json=query, auth=(username, password))

In [ ]:
search_url


'https://es-1.datanusantara.com:9201/news-crawl-production-v2/_search'

In [ ]:
search_results = response.json()
# print(json.dumps(search_results, indent=2))
print(len(search_results['hits']['hits']))

600


In [ ]:
# Parse the response to extract relevant data
search_results = response.json()

# Process the data and create a DataFrame
data = []
for hit in search_results.get("hits", {}).get("hits", []):
    target = hit["_source"]["target"]
    title = hit["_source"]["title"]
    text = hit["_source"]["description_content"]

    data.append({"target": target,  "title": title, "text": text})

# Create a DataFrame
df = pd.DataFrame(data)
print(df.head())

    target                                              title  \
0  banggai  Mobil Sim Keliling Polres Banggai Meriahkan Pa...   
1  banggai  Film Horor “Piara Mayat”,  Kreativitas Anak Mu...   
2  banggai  Festival Malabot Tumbe 2023 Kembali Meriahkan ...   
3  banggai  Rakor Kampanye Damai Pemilu 2024, Polres Bangg...   
4  banggai  Bawaslu Banggai Perkuat Koordinasi dengan Panw...   

                                                text  
0  KABAR LUWUK –  Suasana Pasar Murah bersubsidi ...  
1  KABAR LUWUK  –  Film Horor“Piara Mayat”,Kreati...  
2  KABAR LUWUK  –Festival Malabot Tumbe 2023Kemba...  
3  “Kerja Cepat dan Terkoordinir dalam Penerbitan...  
4  KABAR LUWUK  –Bawaslu BanggaiPerkuat Koordinas...  


In [ ]:
df

,target,title,text
0,banggai,Mobil Sim Keliling Polres Banggai Meriahkan Pa...,KABAR LUWUK – Suasana Pasar Murah bersubsidi ...
1,banggai,"Film Horor “Piara Mayat”, Kreativitas Anak Mu...","KABAR LUWUK – Film Horor“Piara Mayat”,Kreati..."
2,banggai,Festival Malabot Tumbe 2023 Kembali Meriahkan ...,KABAR LUWUK –Festival Malabot Tumbe 2023Kemba...
3,banggai,"Rakor Kampanye Damai Pemilu 2024, Polres Bangg...",“Kerja Cepat dan Terkoordinir dalam Penerbitan...
4,banggai,Bawaslu Banggai Perkuat Koordinasi dengan Panw...,KABAR LUWUK –Bawaslu BanggaiPerkuat Koordinas...
...,...,...,...
595,banggai,Banggai Run Challenge 2024 Menggantikan Tour d...,KABAR LUWUK –Banggai Run Challenge 2024menggan...
596,banggai,"Baru PDIP, Bawaslu Banggai Minta Parpol Segera...",BANGGAI RAYA- Badan Pengawas Pemilu (Bawaslu) ...
597,banggai,Command Center Pemkab Banggai segera Dibangun ...,"Laporan Wartawan TribunPalu.com, Asnawi Zikri ..."
598,banggai,"Rp 4,2 M Disiapkan Pemda Banggai untuk Isentif...","BANGGAI RAYA- Tahun ini, Pemerintah Daerah (Pe..."


In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 600 entries, 0 to 599
Data columns (total 3 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   target  600 non-null    object
 1   title   600 non-null    object
 2   text    600 non-null    object
dtypes: object(3)
memory usage: 14.2+ KB


# Preprocessing

## Delete Duplicates

In [ ]:
df.duplicated(subset='title', keep='first').sum()

20

In [ ]:
df = df.drop_duplicates(subset='title', keep='first')

## Case Folding

In [ ]:
def clean_lower(lwr):
    lwr = lwr.lower() # lowercase text
    return lwr

In [ ]:
# Buat kolom tambahan untuk data ulasan yang telah dicasefolding
df['lwr'] = df['text'].apply(clean_lower)
df['lwr']

0      kabar luwuk –  suasana pasar murah bersubsidi ...
1      kabar luwuk  –  film horor“piara mayat”,kreati...
2      kabar luwuk  –festival malabot tumbe 2023kemba...
3      “kerja cepat dan terkoordinir dalam penerbitan...
4      kabar luwuk  –bawaslu banggaiperkuat koordinas...
                             ...                        
595    kabar luwuk –banggai run challenge 2024menggan...
596    banggai raya- badan pengawas pemilu (bawaslu) ...
597    laporan wartawan tribunpalu.com, asnawi zikri ...
598    banggai raya- tahun ini, pemerintah daerah (pe...
599    terasbanggai.com -sejumlah usulan pembangunan ...
Name: lwr, Length: 580, dtype: object

In [ ]:
df

In [ ]:
print(df['text'][30])
print("_______________________________________________________________________________________________________________")
print(df['lwr'][30])

KABAR LUWUK  –Dinas Sosial BanggaiSegera Salurkan 2.42 MiliarProgram Gercep Gaskan Berdaya2023.Dinas Sosial Kabupaten Banggaimenerima bantuan sebesar 2.42 miliar rupiah dalam program Gerak Cepat Pengentasan Kemiskinan Berbasis Pemberdayaan Masyarakat (GERCEP GASKAN BERDAYA).Bantuan ini ditujukan untuk 242 keluarga penerima manfaat di lima kecamatan, memberikan harapan bagi warga di Kecamatan Luwuk (35 KPM), Luwuk Selatan (28 KPM), Pagimana (95 KPM), Bunta (47 KPM), dan Nuhon (37 KPM). Selasa 28/11/2023.Kepala Dinas Sosial Kabupaten Banggai, Irpan Poma melalui Kepala Bidang Fakir Miskis,Ronald Putje,menyampaikan kepada media bahwa dana bantuan ini merupakan sumbangan langsung dari Pemerintah Provinsi Sulawesi Tengah.Dinas Sosial Kabupaten Banggai bertanggung jawab dalam memfasilitasi distribusi bantuan dan ikut serta dalam pengawasan ketat agar dana tersebut benar-benar sampai kepada penerima manfaat yang tepat.Data yang tersedia menunjukkan bahwa 242 Keluarga Penerima Manfaat yang terd

## Cleansing

### Remove Numbers

In [ ]:
def remove_numbers(text):
  text = re.sub('[0-9]+', '', text)
  return text

In [ ]:
df['clean_number'] = df['lwr'].apply(remove_numbers)
df['clean_number']

0      kabar luwuk –  suasana pasar murah bersubsidi ...
1      kabar luwuk  –  film horor“piara mayat”,kreati...
2      kabar luwuk  –festival malabot tumbe kembali m...
3      “kerja cepat dan terkoordinir dalam penerbitan...
4      kabar luwuk  –bawaslu banggaiperkuat koordinas...
                             ...                        
595    kabar luwuk –banggai run challenge menggantika...
596    banggai raya- badan pengawas pemilu (bawaslu) ...
597    laporan wartawan tribunpalu.com, asnawi zikri ...
598    banggai raya- tahun ini, pemerintah daerah (pe...
599    terasbanggai.com -sejumlah usulan pembangunan ...
Name: clean_number, Length: 580, dtype: object

In [ ]:
df

### Remove Punctuation

In [ ]:
#Remove Puncutuation
clean_spcl = re.compile('[/(){}\[\]\|@,;]')
clean_symbol = re.compile('[^0-9a-z]')
def clean_punct(text):
    text = clean_spcl.sub('', text)
    text = clean_symbol.sub(' ', text)
    return text

In [ ]:
# Buat kolom tambahan untuk data description yang telah diremovepunctuation
df['clean_punct'] = df['clean_number'].apply(clean_punct)
df['clean_punct']

0      kabar luwuk    suasana pasar murah bersubsidi ...
1      kabar luwuk     film horor piara mayat kreativ...
2      kabar luwuk   festival malabot tumbe kembali m...
3       kerja cepat dan terkoordinir dalam penerbitan...
4      kabar luwuk   bawaslu banggaiperkuat koordinas...
                             ...                        
595    kabar luwuk  banggai run challenge menggantika...
596    banggai raya  badan pengawas pemilu bawaslu ka...
597    laporan wartawan tribunpalu com asnawi zikri t...
598    banggai raya  tahun ini pemerintah daerah pemd...
599    terasbanggai com  sejumlah usulan pembangunan ...
Name: clean_punct, Length: 580, dtype: object

In [ ]:
df


### Delete whitespaces

In [ ]:
def normalize_whitespace(text):
    corrected = str(text)
    corrected = re.sub(r"//t",r"\t", corrected)
    corrected = re.sub(r"( )\1+",r"\1", corrected)
    corrected = re.sub(r"(\n)\1+",r"\1", corrected)
    corrected = re.sub(r"(\r)\1+",r"\1", corrected)
    corrected = re.sub(r"(\t)\1+",r"\1", corrected)
    return corrected.strip(" ")

In [ ]:
df['clean_double_ws'] = df['clean_punct'].apply(normalize_whitespace)
df['clean_double_ws']

0      kabar luwuk suasana pasar murah bersubsidi di ...
1      kabar luwuk film horor piara mayat kreativitas...
2      kabar luwuk festival malabot tumbe kembali mer...
3      kerja cepat dan terkoordinir dalam penerbitan ...
4      kabar luwuk bawaslu banggaiperkuat koordinasi ...
                             ...                        
595    kabar luwuk banggai run challenge menggantikan...
596    banggai raya badan pengawas pemilu bawaslu kab...
597    laporan wartawan tribunpalu com asnawi zikri t...
598    banggai raya tahun ini pemerintah daerah pemda...
599    terasbanggai com sejumlah usulan pembangunan d...
Name: clean_double_ws, Length: 580, dtype: object

In [ ]:
df

## Stopword Removal

### Get custom stopwords

In [ ]:
url = 'https://raw.githubusercontent.com/TaillessTanuki/ID-Stopwords/main/ID-Stopwords(MasDevid).csv'
df_stopword = pd.read_csv(url, delimiter = ";",encoding='cp1252')

In [ ]:
df_stopword.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 758 entries, 0 to 757
Data columns (total 1 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   Stopwords  758 non-null    object
dtypes: object(1)
memory usage: 6.0+ KB


In [ ]:
df_stopword

,Stopwords
0,ada
1,adalah
2,adanya
3,adapun
4,agak
...,...
753,wong
754,yaitu
755,yakin
756,yakni


In [ ]:
custom_stopword = list(df_stopword['Stopwords'])
custom_stopword

### Removal

In [ ]:
# Prepare stopword remover
factory = StopWordRemoverFactory()
stopwords = factory.get_stop_words() + custom_stopword
print(stopwords)

['yang', 'untuk', 'pada', 'ke', 'para', 'namun', 'menurut', 'antara', 'dia', 'dua', 'ia', 'seperti', 'jika', 'jika', 'sehingga', 'kembali', 'dan', 'tidak', 'ini', 'karena', 'kepada', 'oleh', 'saat', 'harus', 'sementara', 'setelah', 'belum', 'kami', 'sekitar', 'bagi', 'serta', 'di', 'dari', 'telah', 'sebagai', 'masih', 'hal', 'ketika', 'adalah', 'itu', 'dalam', 'bisa', 'bahwa', 'atau', 'hanya', 'kita', 'dengan', 'akan', 'juga', 'ada', 'mereka', 'sudah', 'saya', 'terhadap', 'secara', 'agar', 'lain', 'anda', 'begitu', 'mengapa', 'kenapa', 'yaitu', 'yakni', 'daripada', 'itulah', 'lagi', 'maka', 'tentang', 'demi', 'dimana', 'kemana', 'pula', 'sambil', 'sebelum', 'sesudah', 'supaya', 'guna', 'kah', 'pun', 'sampai', 'sedangkan', 'selagi', 'sementara', 'tetapi', 'apakah', 'kecuali', 'sebab', 'selain', 'seolah', 'seraya', 'seterusnya', 'tanpa', 'agak', 'boleh', 'dapat', 'dsb', 'dst', 'dll', 'dahulu', 'dulunya', 'anu', 'demikian', 'tapi', 'ingin', 'juga', 'nggak', 'mari', 'nanti', 'melainkan', '

In [ ]:
len(stopwords)

884

In [ ]:
dictionary = ArrayDictionary(stopwords)
stopword = StopWordRemover(dictionary)

In [ ]:
def filtering(text):
  stop = stopword.remove(text)
  return stop

In [ ]:
df

In [ ]:
df['filtered'] = df['clean_double_ws'].apply(filtering)
df['filtered']

0      kabar luwuk suasana pasar murah bersubsidi kab...
1      kabar luwuk film horor piara mayat kreativitas...
2      kabar luwuk festival malabot tumbe meriahkanka...
3      kerja cepat terkoordinir penerbitan sttp kabar...
4      kabar luwuk bawaslu banggaiperkuat koordinasi ...
                             ...                        
595    kabar luwuk banggai run challenge menggantikan...
596    banggai raya badan pengawas pemilu bawaslu kab...
597    laporan wartawan tribunpalu com asnawi zikri t...
598    banggai raya pemerintah daerah pemda kabupaten...
599    terasbanggai com usulan pembangunan setiap wil...
Name: filtered, Length: 580, dtype: object

In [ ]:
df

In [ ]:
len(df)

580

In [ ]:
print(df['clean_double_ws'][69])
print(df['filtered'][69])

kabar luwuk ratusan buruh kepung dinas koperasidan kupp luwuk hingga akhirnya mencapai kesepakatan menjadi saksi dari aksi protes massal yang melibatkan ratusan buruhtangkiang mahasiswa dan masyarakat yang berkumpul untuk mendesak pembatalan pernyataan kontroversial plh kepala kupp kelas ii luwuk nukman larau rabu november kontroversi dimulai setelah rapat forkopimda banggai pada senin november di mana nukman larau mengakomodir kepengurusan baruuupj tangkiang yang dikeluarkan oleh koperasi tkbm teluk lalong perubahan kepengurusan ini mencakup penggantian ketua dan beberapa pengurus dengan orang yang bukan anggota uupj tangkiang protes muncul ketika sk pergantian kepengurusan ditolak oleh seluruh anggota uupj tangkiang dalam rapat luar biasa mereka tetap memilih amir mangulele sebagai ketua menyatakan penolakan terhadap upaya mengadu domba buruh yang dianggap dilakukan oleh koperasi tkbm teluk lalong selanjutnya massa aksi memulai pengepungan kantor dinas koperasi menuntut agar nukman l

## Stemming

In [ ]:
factory = StemmerFactory()
stemmer = factory.create_stemmer()

In [ ]:
def stem(text):
  return stemmer.stem(text)

In [ ]:
df['stemmed'] = df['filtered'].apply(stem)
df['stemmed']

0      kabar luwuk suasana pasar murah subsidi kabupa...
1      kabar luwuk film horor piara mayat kreativitas...
2      kabar luwuk festival malabot tumbe meriahkanka...
3      kerja cepat terkoordinir terbit sttp kabar luw...
4      kabar luwuk bawaslu banggaiperkuat koordinasi ...
                             ...                        
595    kabar luwuk bangga run challenge menggantikant...
596    bangga raya badan awas milu bawaslu kabupaten ...
597    lapor wartawan tribunpalu com asnawi zikri tri...
598    bangga raya perintah daerah pemda kabupaten ba...
599    terasbanggai com usul bangun tiap wilayah desa...
Name: stemmed, Length: 580, dtype: object

In [ ]:
df['stemmed'].to_csv("stemmed_words.csv", index=False)

In [ ]:
print(df['filtered'][100])
print(df['stemmed'][100])

kabar luwuk prestasi kejaksaan negeri banggai berkibar bidang intelijen kantor kejaksaan negeri kabupaten banggai menunjukkan keunggulan bidang intelijen bawah kepemimpinan raden bagus wicaksono sh m hum kasi intelijen berhasil meraih penghargaan terbaik sebuah prestasi gemilang diberikan kepala kejaksaan sulawesi prestasi dipandang pengakuan kualitas pelaporan administrasi intelijen kejaksaan negeri banggai raden bagus wicaksono rasa bangganya pencapaian menggambarkan komitmen kuat profesionalisme ketelitian menjalankan tugas intelijen ungkapannya kepala kejaksaan negeri banggai terima kasih kejaksaan sulawesi ia berjanji terus meningkatkan kinerja kejaksaan negeri banggai menjaga tingginya standar bidang intelijen prestasi tidak mencerminkan kesuksesan individu juga semangat kolaboratif tim kejaksaan negeri banggai pencapaian juga menyoroti dedikasi kepala seksi intelijen kabupaten banggai sarman santosa tandisau sh secara langsung menerima penghargaan keberhasilan bukan prestasi for

## Tokenizing

In [ ]:
def tokenize(text):
  tokens = nltk.tokenize.word_tokenize(text)
  return tokens

In [ ]:
df['tokenized'] = df['stemmed'].apply(tokenize)
df['tokenized']

0      [kabar, luwuk, suasana, pasar, murah, subsidi,...
1      [kabar, luwuk, film, horor, piara, mayat, krea...
2      [kabar, luwuk, festival, malabot, tumbe, meria...
3      [kerja, cepat, terkoordinir, terbit, sttp, kab...
4      [kabar, luwuk, bawaslu, banggaiperkuat, koordi...
                             ...                        
595    [kabar, luwuk, bangga, run, challenge, menggan...
596    [bangga, raya, badan, awas, milu, bawaslu, kab...
597    [lapor, wartawan, tribunpalu, com, asnawi, zik...
598    [bangga, raya, perintah, daerah, pemda, kabupa...
599    [terasbanggai, com, usul, bangun, tiap, wilaya...
Name: tokenized, Length: 580, dtype: object

In [ ]:
df

,target,title,text,lwr,clean_number,clean_punct,clean_double_ws,filtered,stemmed,tokenized
0,banggai,Mobil Sim Keliling Polres Banggai Meriahkan Pa...,KABAR LUWUK – Suasana Pasar Murah bersubsidi ...,kabar luwuk – suasana pasar murah bersubsidi ...,kabar luwuk – suasana pasar murah bersubsidi ...,kabar luwuk suasana pasar murah bersubsidi ...,kabar luwuk suasana pasar murah bersubsidi di ...,kabar luwuk suasana pasar murah bersubsidi kab...,kabar luwuk suasana pasar murah subsidi kabupa...,"[kabar, luwuk, suasana, pasar, murah, subsidi,..."
1,banggai,"Film Horor “Piara Mayat”, Kreativitas Anak Mu...","KABAR LUWUK – Film Horor“Piara Mayat”,Kreati...","kabar luwuk – film horor“piara mayat”,kreati...","kabar luwuk – film horor“piara mayat”,kreati...",kabar luwuk film horor piara mayat kreativ...,kabar luwuk film horor piara mayat kreativitas...,kabar luwuk film horor piara mayat kreativitas...,kabar luwuk film horor piara mayat kreativitas...,"[kabar, luwuk, film, horor, piara, mayat, krea..."
2,banggai,Festival Malabot Tumbe 2023 Kembali Meriahkan ...,KABAR LUWUK –Festival Malabot Tumbe 2023Kemba...,kabar luwuk –festival malabot tumbe 2023kemba...,kabar luwuk –festival malabot tumbe kembali m...,kabar luwuk festival malabot tumbe kembali m...,kabar luwuk festival malabot tumbe kembali mer...,kabar luwuk festival malabot tumbe meriahkanka...,kabar luwuk festival malabot tumbe meriahkanka...,"[kabar, luwuk, festival, malabot, tumbe, meria..."
3,banggai,"Rakor Kampanye Damai Pemilu 2024, Polres Bangg...",“Kerja Cepat dan Terkoordinir dalam Penerbitan...,“kerja cepat dan terkoordinir dalam penerbitan...,“kerja cepat dan terkoordinir dalam penerbitan...,kerja cepat dan terkoordinir dalam penerbitan...,kerja cepat dan terkoordinir dalam penerbitan ...,kerja cepat terkoordinir penerbitan sttp kabar...,kerja cepat terkoordinir terbit sttp kabar luw...,"[kerja, cepat, terkoordinir, terbit, sttp, kab..."
4,banggai,Bawaslu Banggai Perkuat Koordinasi dengan Panw...,KABAR LUWUK –Bawaslu BanggaiPerkuat Koordinas...,kabar luwuk –bawaslu banggaiperkuat koordinas...,kabar luwuk –bawaslu banggaiperkuat koordinas...,kabar luwuk bawaslu banggaiperkuat koordinas...,kabar luwuk bawaslu banggaiperkuat koordinasi ...,kabar luwuk bawaslu banggaiperkuat koordinasi ...,kabar luwuk bawaslu banggaiperkuat koordinasi ...,"[kabar, luwuk, bawaslu, banggaiperkuat, koordi..."
...,...,...,...,...,...,...,...,...,...,...
595,banggai,Banggai Run Challenge 2024 Menggantikan Tour d...,KABAR LUWUK –Banggai Run Challenge 2024menggan...,kabar luwuk –banggai run challenge 2024menggan...,kabar luwuk –banggai run challenge menggantika...,kabar luwuk banggai run challenge menggantika...,kabar luwuk banggai run challenge menggantikan...,kabar luwuk banggai run challenge menggantikan...,kabar luwuk bangga run challenge menggantikant...,"[kabar, luwuk, bangga, run, challenge, menggan..."
596,banggai,"Baru PDIP, Bawaslu Banggai Minta Parpol Segera...",BANGGAI RAYA- Badan Pengawas Pemilu (Bawaslu) ...,banggai raya- badan pengawas pemilu (bawaslu) ...,banggai raya- badan pengawas pemilu (bawaslu) ...,banggai raya badan pengawas pemilu bawaslu ka...,banggai raya badan pengawas pemilu bawaslu kab...,banggai raya badan pengawas pemilu bawaslu kab...,bangga raya badan awas milu bawaslu kabupaten ...,"[bangga, raya, badan, awas, milu, bawaslu, kab..."
597,banggai,Command Center Pemkab Banggai segera Dibangun ...,"Laporan Wartawan TribunPalu.com, Asnawi Zikri ...","laporan wartawan tribunpalu.com, asnawi zikri ...","laporan wartawan tribunpalu.com, asnawi zikri ...",laporan wartawan tribunpalu com asnawi zikri t...,laporan wartawan tribunpalu com asnawi zikri t...,laporan wartawan tribunpalu com asnawi zikri t...,lapor wartawan tribunpalu com asnawi zikri tri...,"[lapor, wartawan, tribunpalu, com, asnawi, zik..."
598,banggai,"Rp 4,2 M Disiapkan Pemda Banggai untuk Isentif...","BANGGAI RAYA- Tahun ini, Pemerintah Daerah (Pe...","banggai raya- tahun ini, pemerintah daerah (pe...","bangga

In [ ]:
print(df['stemmed'][264])
print(df['tokenized'][264])

lapor nasri seibanggaipostbanggaipost comluwuk jaksa negeri kejar bangga tangan korupsi daerah dasar rilis sampai kasi intelejen sarman santosa tandisau s h jumat perkara korupsi antara proses sidi yaknipertama duga tindak pidana korupsi kerja rekonstruksi talud kam pantai desa gorontalo camat balantak selatan anggar duga tindak pidana korupsi penyalahgunaan uang serta modal badan usaha milik desa bumdes berkah uso desa uso camat batu kabupaten bangga selain satu perkara tahap tuntut duga perkara tindak pidana korupsi penyalahgunaan apbdesa lobu t a t a an dakwa lusiana udopo langgar primair pasal ayat jo pasal ayat undang undang no tentang berantas tindak pidana korupsi telah ubah tambah undang undang no tentang ubah undang undang no tentang berantas tindak pidana korupsi subsidair pasal jo pasal ayat undang undang no tentang berantas tindak pidana korupsi kara a quo upaya hukum kasasi mahkamah agung sarman kejar bangga telah eksekusi kasus korupsi yaknipertama duga tindak pidana koru

# Convert df to excel

In [ ]:
df.to_excel('lexicon_analytics_banggai_news.xlsx', index=False)